<a href="https://colab.research.google.com/github/noaihere/encyclepedia_python/blob/main/cna_covid_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download covid data and save to local csv

In [ ]:
import pandas as pd
from google.colab import files
import json
import requests

In [ ]:
# get data needed for charts in 
# https://www.channelnewsasia.com/news/singapore/covid-19-singapore-new-community-case-restaurant-manager-14646412

session = requests.Session()
response = session.get("https://infographics.channelnewsasia.com/covid-19/sgsitrep.json").json()
print(response.keys())
data = response['feed']["entry"]

dict_keys(['version', 'encoding', 'feed'])


In [ ]:
df = pd.DataFrame(data)
df.describe()
df.head(2)

,id,updated,category,title,content,link,gsx$date,gsx$totalcases,gsx$communitycases,gsx$dormresidents,gsx$importedcases,gsx$workpermitholdersnotindorms,gsx$new,gsx$correct,gsx$updatedasof,gsx$embeddedstartdate,gsx$totallocalnon-dormcases,gsx$highestnumber
0,{'$t': 'https://spreadsheets.google.com/feeds/...,{'$t': '2021-04-26T02:42:01.875Z'},[{'scheme': 'http://schemas.google.com/spreads...,"{'type': 'text', '$t': 'Row: 2'}","{'type': 'text', '$t': 'updatedasof: Apr 25, 3...","[{'rel': 'self', 'type': 'application/atom+xml...",{'$t': ''},{'$t': ''},{'$t': ''},{'$t': ''},{'$t': ''},{'$t': ''},{'$t': ''},{'$t': ''},"{'$t': 'Apr 25, 3.50pm'}",{'$t': 'Mar 15'},{'$t': ''},{'$t': '66'}
1,{'$t': 'https://spreadsheets.google.com/feeds/...,{'$t': '2021-04-26T02:42:01.875Z'},[{'scheme': 'http://schemas.google.com/spreads...,"{'type': 'text', '$t': 'Apr 1'}","{'type': 'text', '$t': 'totalcases: 74, commun...","[{'rel': 'self', 'type': 'application/atom+xml...",{'$t': 'Apr 1'},{'$t': '74'},{'$t': '39'},{'$t': '13'},{'$t': '19'},{'$t': '3'},{'$t': ''},{'$t': 'YES'},{'$t': ''},{'$t': ''},{'$t': '42'},{'$t': ''}


In [ ]:
# get number values only using str extract

list_col = ['gsx$communitycases', 'gsx$dormresidents', 'gsx$importedcases']
for col in list_col:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.extract('(\d+)')
    
df.head(2)

,id,updated,category,title,content,link,gsx$date,gsx$totalcases,gsx$communitycases,gsx$dormresidents,gsx$importedcases,gsx$workpermitholdersnotindorms,gsx$new,gsx$correct,gsx$updatedasof,gsx$embeddedstartdate,gsx$totallocalnon-dormcases,gsx$highestnumber
0,{'$t': 'https://spreadsheets.google.com/feeds/...,{'$t': '2021-04-26T02:42:01.875Z'},[{'scheme': 'http://schemas.google.com/spreads...,"{'type': 'text', '$t': 'Row: 2'}","{'type': 'text', '$t': 'updatedasof: Apr 25, 3...","[{'rel': 'self', 'type': 'application/atom+xml...",{'$t': ''},{'$t': ''},NaN,NaN,NaN,{'$t': ''},{'$t': ''},{'$t': ''},"{'$t': 'Apr 25, 3.50pm'}",{'$t': 'Mar 15'},{'$t': ''},{'$t': '66'}
1,{'$t': 'https://spreadsheets.google.com/feeds/...,{'$t': '2021-04-26T02:42:01.875Z'},[{'scheme': 'http://schemas.google.com/spreads...,"{'type': 'text', '$t': 'Apr 1'}","{'type': 'text', '$t': 'totalcases: 74, commun...","[{'rel': 'self', 'type': 'application/atom+xml...",{'$t': 'Apr 1'},{'$t': '74'},39,13,19,{'$t': '3'},{'$t': ''},{'$t': 'YES'},{'$t': ''},{'$t': ''},{'$t': '42'},{'$t': ''}


In [ ]:
# extract date and assign year 2021

df['gsx$date'] = df['gsx$date'].astype(str)
df['gsx$date'] = df['gsx$date'].str.extract("(\w+ \d+)")
df['gsx$date'] = '2021 ' + df['gsx$date']
df['gsx$date'].head(2)

0           NaN
1    2021 Apr 1
Name: gsx$date, dtype: object

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
# make to datetime col

df['gsx$date'] = pd.to_datetime(df['gsx$date'], format = '%Y %b %d')
df['gsx$date'].head(3)

0          NaT
1   2021-04-01
2   2021-04-02
Name: gsx$date, dtype: datetime64[ns]

In [ ]:
# get required cols

final = df[['gsx$date','gsx$communitycases', 'gsx$dormresidents', 'gsx$importedcases']]

# rename cols
final.columns = final.columns.str.replace('gsx\$','')

# remove all na rows
final = final.dropna(how='all')

final.head()

,date,communitycases,dormresidents,importedcases
1,2021-04-01,39,13,19
2,2021-04-02,29,8,8
3,2021-04-03,29,22,9
4,2021-04-04,32,26,7
5,2021-04-05,43,58,4


In [ ]:
final.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,date,communitycases,dormresidents,importedcases
count,390,390,390,390
unique,365,38,166,43
top,2021-04-03 00:00:00,0,0,0
freq,2,151,169,77
first,2021-01-01 00:00:00,NaN,NaN,NaN
last,2021-12-31 00:00:00,NaN,NaN,NaN


In [ ]:
# not all dates from 2021

final[final['date']==pd.to_datetime('2021-12-31')]

,date,communitycases,dormresidents,importedcases
275,2021-12-31,5,0,25


In [ ]:
# see rows before dec 31
# looks like below to 2020 data
print(final.iloc[:275,0])

# get dec 31 row index
dec_index = final[final['date']==pd.to_datetime('2021-12-31')].index[0]
dec_index

1     2021-04-01
2     2021-04-02
3     2021-04-03
4     2021-04-04
5     2021-04-05
         ...    
271   2021-12-27
272   2021-12-28
273   2021-12-29
274   2021-12-30
275   2021-12-31
Name: date, Length: 275, dtype: datetime64[ns]


275

In [ ]:
# minus 1 year from date if row index is before dec 31 row index

final.iloc[:dec_index,0]= final.iloc[:dec_index,0]- pd.DateOffset(years=1)
final.iloc[:dec_index,0]

1     2020-04-01
2     2020-04-02
3     2020-04-03
4     2020-04-04
5     2020-04-05
         ...    
271   2020-12-27
272   2020-12-28
273   2020-12-29
274   2020-12-30
275   2020-12-31
Name: date, Length: 275, dtype: datetime64[ns]

In [ ]:
list_col = ['communitycases', 'dormresidents', 'importedcases']
for col in list_col:
    final[col] = final[col].astype(int)

In [ ]:
final2021= final[final['date']>pd.to_datetime('2021-1-1')]
final2021.describe()

,communitycases,dormresidents,importedcases
count,114.000000,114.000000,114.000000
mean,0.684211,0.087719,20.078947
std,1.115584,0.284135,10.577271
min,0.000000,0.000000,1.000000
25%,0.000000,0.000000,12.000000
50%,0.000000,0.000000,17.500000
75%,1.000000,0.000000,26.000000
max,5.000000,1.000000,55.000000


In [ ]:
final2021.to_csv('final2021.csv', index=False)
files.download('final2021.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>